In [1]:
import numpy as np
import pandas as pd
from os import getcwd
import glob
from os.path import join
import ntpath
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import pickle

import warnings
import multiprocessing 

In [2]:
def set_path(): 
   abspath = getcwd()
   dname = os.path.dirname(abspath)
   os.chdir(dname)

set_path()
 

In [3]:
def score_NLP(auth_NLP,vectorizer,testcase):#all words in train dictionary of one authorb
    auth_nlp = auth_NLP.toarray()
    testTF = vectorizer.transform([testcase]).toarray()[0]
    score = []
    for article in auth_nlp:
        score.append(np.dot(article, testTF))
    return max(score)




In [4]:
def score_Place(placeMod, testcase):
    score = placeMod.predict_proba([testcase])[0][1]
    return score

In [5]:
def calculate_final_score(score_nlp, score_place):
   final_score = score_nlp + score_place 
   return final_score

In [6]:
# prediction for one case:
def prediction(year, venue,content):
   # preparing test cases: 
   testcase_NLP = content
   #testcase_place = [year, venue]

   # access archive:

   pkl_file_list = glob.glob(join(getcwd(),'models_TF',"*.pkl"))
   result_records = {}
   for fname in pkl_file_list:
      _, authid = ntpath.split(fname)
      authid = int(authid[:-4])
      with open(fname, 'rb') as f:
         models = pickle.load(file = f)
      
      # NLP score
      auth_NLP, vectorizer = models[0] 
      score_nlp = score_NLP(auth_NLP, vectorizer,testcase_NLP)

      #place score
      #place_model = models[1]
      #score_place = 0           ############## Remember to change `model_place`` later

      final_score =  score_nlp      ## `calculate_final_score(score_nlp, score_place)`
      result_records[authid] = final_score
   return max(result_records, key=result_records.get)
      


In [7]:
# predict all 

def predict_all(df):
   pred = []
   for i in range(len(df)):
      if i%100 == 0:
         print(f'case prediction {i}')
      case = df.iloc[i]
      year = case["year"]
      venue = case["venues_le"]
      content = case["content"]
      result = prediction(year, venue, content)
      pred.append(result)
   return pred

      

In [8]:
warnings.filterwarnings('ignore')
df_val= pd.read_pickle('data/processed/val_clean_df.pkl')

val_pred = predict_all(df = df_val)

case prediction 0
case prediction 100


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df_val['authId_enc'].values, np.array(val_pred))

0.0

#### parallel code: